# Morphological profiling of BiTE coculture images

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import javabridge
import bioformats as bf
import pandas as pd
import seaborn as sn
import random
import sys
import os
import re
import csv

from bioimg import load_image_series

javabridge.start_vm(class_path=bf.JARS)

In [ ]:
path = '/Volumes/gitlab/microscopy/data/Tobias/newscreen/'
plate = 'Tag3'
for f in os.listdir(path):
    if re.search(plate + "_", f):
        screen_id = f
platedir = os.path.join(path, screen_id, 'Images')

In [ ]:
fnames = [f for f in os.listdir(platedir) if '.tiff' in f]

In [ ]:
all_wells = list(set([re.search('r[0-9]+c[0-9]+',f).group(0) for f in fnames]))
all_wells.sort()

In [ ]:
well_id = 0
well = all_wells[well_id]
well_imgs = [f for f in fnames if well in f and 'ch2' not in f]

In [ ]:
imglist = []
for i in range(1,5):
    wellpos = [f for f in well_imgs if 'f0' + str(i)+ 'p' in f]
    imgseries = load_image_series(path=platedir, imgfiles=wellpos)
    imgseries = imgseries.reshape((8, 4, 2160,2160))
    mipseries = np.amax(imgseries, axis=0)
    imglist.append(mipseries)

In [ ]:
len(imglist)

In [ ]:
imglist[0].shape

In [ ]:
col_params = dict(colors=['blue', 'red', 
                          'yellow', 'green'],
                             blend = [2, 0.8, 0.8, 0.8],
                             gamma = [0.4, 0.3, 0.3, 0.3])

In [ ]:
from bioimg import combine_channels
mip_color = combine_channels([imglist[3][i] for i in range(4)],
                             **col_params)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(mip_color[:1000,:1000,:])
plt.axis('off')

## Segment B and T cell nuclei

In [ ]:
hoechst = imglist[3][0]

In [ ]:
from skimage.feature import shape_index, blob_log
from bioimg import threshold_img

def nantonum(img, pad=-1):
    img_r = np.copy(img)
    img_r[np.isnan(img_r)] = pad
    return img_r

In [ ]:
gamma = 0.5
img_th = threshold_img(hoechst**gamma, method='otsu', binary=False)
img_s = shape_index(img_th)
img_enh = nantonum(img_s, pad=-1)

In [ ]:
# run LoG blob detection on the shape-index enhanced image
blobs_enh = blob_log(img_enh,
                 min_sigma=10, max_sigma=14, threshold=0.05)

In [ ]:
'''fig, ax = plt.subplots(figsize=(10,10))
for blob in blobs_enh:
    y, x, r = blob
    c = plt.Circle((x, y), r+2, color='magenta', linewidth=1.4, fill=False)
    ax.add_patch(c)
ax.imshow(hoechst**0.5, cmap='gray')
ax.axis('off')'''

Convert "blobs" to bounding boxes:

In [ ]:
pad = 5
bbox = np.stack([np.array([bl[1] - bl[2] - pad,
                           bl[1] + bl[2] + pad,
                           bl[0] - bl[2] - pad,
                           bl[0] + bl[2] + pad]) for bl in blobs_enh])

In [ ]:
imax = hoechst.shape[0] - 1

In [ ]:
bbox[bbox < 0] = 0
bbox[bbox > imax ] = imax

In [ ]:
bbox = bbox.astype(int)

In [ ]:
from bioimg import show_bbox
show_bbox(img=mip_color[:1000,:1000,:], bbox=bbox)

In [ ]:
from bioimg import ImgX
# initialize 'ImgX' class
imgx = ImgX(img=imglist[3].swapaxes(0,-1), 
            bbox=bbox,
            n_chan=['Hoechst', 'PE',
                    'APC', 'Calcein'])

In [ ]:
imgx.params['texture'] = 'both'
imgx.compute_props()
img_df = imgx.get_df().copy()

In [ ]:
img_df.shape

In [ ]:
img_df.head()

In [ ]:
sn.distplot(img_df['ch-Calcein-mean_intensity'], kde=False)

## Spatial Cluster Quantification
Threshold Calcein channel and perform morphological dilation to find clusters:

In [ ]:
# get calcein channel
ca = imglist[3][3]

plt.figure(figsize=(8,8))
plt.imshow(ca**0.7)
plt.axis('off')

This well image doesn't have any clusters. However, apply morphological dilation and check the behavior:

In [ ]:
# threshold the Calcein channel
ca_th = threshold_img(ca, method='otsu', binary=True)

plt.figure(figsize=(8,8))
plt.imshow(ca_th)
plt.axis('off')

In [ ]:
from skimage.morphology import binary_dilation, disk

ca_dil = binary_dilation(ca_th, disk(5))
plt.figure(figsize=(8,8))
plt.imshow(ca_dil)
plt.axis('off')

Next only select objects with Calcein area > 100, i.e. the identified "clusters" have to be at least larger than a typical viable cell

In [ ]:
from skimage.measure import label
from skimage.color import label2rgb
from bioimg.segment.cv_methods import filter_segm

In [ ]:
bounds = {'area': (1400, np.inf)}
segm = filter_segm(img=ca,
                    labels=label(ca_dil, connectivity=1),
                    bounds=bounds)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(label2rgb(segm, image=ca, bg_label=0))
ax.axis('off')

Now load a different well -- the one that has some clusters:

In [ ]:
well_id = 10
well = all_wells[well_id]
well_imgs = [f for f in fnames if well in f and 'ch2' not in f]

wellpos = [f for f in well_imgs if 'f02' + 'p' in f]
imgseries = load_image_series(path=platedir, imgfiles=wellpos)
imgseries = imgseries.reshape((8, 4, 2160,2160))
mip = np.amax(imgseries, axis=0)

In [ ]:
# get calcein channel
ca = mip[3]

plt.figure(figsize=(8,8))
plt.imshow(ca**0.7)
plt.axis('off')

In [ ]:
# threshold the Calcein channel
ca_th = threshold_img(ca, method='otsu', binary=True)

plt.figure(figsize=(8,8))
plt.imshow(ca_th)
plt.axis('off')

In [ ]:
ca_dil = binary_dilation(ca_th, disk(2))
plt.figure(figsize=(8,8))
plt.imshow(ca_dil)
plt.axis('off')

In [ ]:
bounds = {'area': (600, np.inf)}
segm = filter_segm(img=ca,
                    labels=label(ca_dil, connectivity=1),
                    bounds=bounds)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
ax.imshow(label2rgb(segm, image=ca, bg_label=0))
ax.axis('off')

In [ ]:
from skimage.measure import regionprops_table
feats = regionprops_table(segm,
                       intensity_image=ca,
                      properties=['bbox'])

In [ ]:
df = pd.DataFrame(feats)
df = df.rename(columns={'bbox-0': 'ymin',
                   'bbox-1': 'xmin',
                   'bbox-2': 'ymax',
                   'bbox-3': 'xmax'})

In [ ]:
df.head()

In [ ]:
rmax, cmax = ca.shape

In [ ]:
from bioimg import read_bbox, show_bbox
bbox = read_bbox(df=df, rmax=rmax, cmax=cmax, pad=0)

In [ ]:
show_bbox(ca, bbox)

In [ ]:
imgx = ImgX(img=mip.swapaxes(0,-1), 
            bbox=bbox,
            n_chan=['Hoechst', 'PE',
                    'APC', 'Calcein'])

In [ ]:
imgx.compute_props()

In [ ]:
img_df = imgx.get_df().copy()

In [ ]:
img_df.shape